#### Пермяков Дмитрий ИУ5-63Б
Вариант 13

## 1. Предобработка данных

In [16]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('marvel-wikia-data.csv')

# Проверка первых строк
print(data.head())

# Проверка информации о данных
print(data.info())


   page_id                                 name  \
0     1678            Spider-Man (Peter Parker)   
1     7139      Captain America (Steven Rogers)   
2    64786  Wolverine (James \"Logan\" Howlett)   
3     1868    Iron Man (Anthony \"Tony\" Stark)   
4     2460                  Thor (Thor Odinson)   

                                   urlslug                ID  \
0              \/Spider-Man_(Peter_Parker)   Secret Identity   
1        \/Captain_America_(Steven_Rogers)   Public Identity   
2  \/Wolverine_(James_%22Logan%22_Howlett)   Public Identity   
3    \/Iron_Man_(Anthony_%22Tony%22_Stark)   Public Identity   
4                    \/Thor_(Thor_Odinson)  No Dual Identity   

                ALIGN         EYE        HAIR              SEX  GSM  \
0     Good Characters  Hazel Eyes  Brown Hair  Male Characters  NaN   
1     Good Characters   Blue Eyes  White Hair  Male Characters  NaN   
2  Neutral Characters   Blue Eyes  Black Hair  Male Characters  NaN   
3     Good Characters   

In [17]:
# Удаление дубликатов столбцов
data = data.loc[:,~data.columns.duplicated()]

# Проверка после удаления дубликатов
print(data.head())

   page_id                                 name  \
0     1678            Spider-Man (Peter Parker)   
1     7139      Captain America (Steven Rogers)   
2    64786  Wolverine (James \"Logan\" Howlett)   
3     1868    Iron Man (Anthony \"Tony\" Stark)   
4     2460                  Thor (Thor Odinson)   

                                   urlslug                ID  \
0              \/Spider-Man_(Peter_Parker)   Secret Identity   
1        \/Captain_America_(Steven_Rogers)   Public Identity   
2  \/Wolverine_(James_%22Logan%22_Howlett)   Public Identity   
3    \/Iron_Man_(Anthony_%22Tony%22_Stark)   Public Identity   
4                    \/Thor_(Thor_Odinson)  No Dual Identity   

                ALIGN         EYE        HAIR              SEX  GSM  \
0     Good Characters  Hazel Eyes  Brown Hair  Male Characters  NaN   
1     Good Characters   Blue Eyes  White Hair  Male Characters  NaN   
2  Neutral Characters   Blue Eyes  Black Hair  Male Characters  NaN   
3     Good Characters   

In [18]:
# Заполнение пропущенных значений
data = data.fillna(method='ffill').fillna(method='bfill')  # Простой способ, можно уточнить в зависимости от данных

# Проверка пропущенных значений
print(data.isnull().sum())

page_id             0
name                0
urlslug             0
ID                  0
ALIGN               0
EYE                 0
HAIR                0
SEX                 0
GSM                 0
ALIVE               0
APPEARANCES         0
FIRST APPEARANCE    0
Year                0
dtype: int64


Кодирование категориальных признаков
Преобразуем категориальные признаки в числовые.

In [19]:
from sklearn.preprocessing import LabelEncoder

# Применение LabelEncoder к категориальным признакам
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Проверка преобразованных данных
print(data.head())

   page_id   name  urlslug  ID  ALIGN  EYE  HAIR  SEX  GSM  ALIVE  \
0     1678  13954    13953   3      1    9     6    3    0      1   
1     7139   2327     2330   2      1    3    23    3    0      1   
2    64786  15996    15999   2      2    3     2    3    0      1   
3     1868   6771     6774   2      1    3     2    3    0      1   
4     2460  14709    14709   1      1    3     3    3    0      1   

   APPEARANCES  FIRST APPEARANCE    Year  
0       4043.0                99  1962.0  
1       3360.0               503  1941.0  
2       3061.0               738  1974.0  
3       2961.0               521  1963.0  
4       2258.0               649  1950.0  


## 2. Разделение данных на обучающую и тестовую выборки
Разделим данные на признаки (X) и целевую переменную (y), а затем на обучающую и тестовую выборки.

In [20]:
from sklearn.model_selection import train_test_split

# Предположим, что последний столбец — это целевая переменная
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка размеров выборок
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13100, 12) (3276, 12) (13100,) (3276,)


## 3. Построение моделей

#### Дерево решений

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

# Создание и обучение модели
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_dt = dt_model.predict(X_test)

# Оценка качества модели
accuracy_dt = accuracy_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')

print(f"Decision Tree Accuracy: {accuracy_dt}")
print(f"Decision Tree F1 Score: {f1_dt}")

Decision Tree Accuracy: 0.3904151404151404
Decision Tree F1 Score: 0.3925889200747888


#### Случайный лес

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Создание и обучение модели
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_rf = rf_model.predict(X_test)

# Оценка качества модели
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest Accuracy: {accuracy_rf}")
print(f"Random Forest F1 Score: {f1_rf}")

Random Forest Accuracy: 0.33485958485958484
Random Forest F1 Score: 0.3175576456271747


## 4. Оценка качества моделей
Мы используем accuracy (точность) и F1 score (F1-мера) для оценки качества моделей:

- Точность: Доля правильно предсказанных наблюдений к общему числу наблюдений.
- F1-мера: Среднее гармоническое точности и полноты. Полезно, когда важен баланс между этими метриками.

### Выводы
#### Дерево решений:
- Accuracy: указывает на долю верных предсказаний.
- F1 Score: полезно для оценки, когда классы несбалансированы.
    
#### Случайный лес:
Обычно показывает лучшую производительность за счет уменьшения переобучения.
Если F1-мера выше у одной модели по сравнению с другой, это указывает на лучшее качество классификации, особенно при несбалансированных данных.

Таким образом, вы сможете выбрать модель, которая показывает наилучшие результаты по данным метрикам на вашем наборе данных.